# Gauss_Matrix Finite Field

This script use the Gaussian pivots to solve matrices and find a polynomial from points in a modular field.

For more information about how it is used in Shamir's secret-sharing scheme, please read [README.md](https://github.com/NoamBaum1/Shamir/blob/main/README.md) or [Shamir.ipynb](https://github.com/NoamBaum1/Shamir/blob/main/Shamir.ipynb)


First the script take the data in *data.txt*, put them in the finite field and put all the coordinates in the list ``pts``like : $[x_1,y_1,y_1,x_2,y_2,x_3,y_3\dots]$

```
fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
a = fichier.read()
fichier.close()
pts = []
coordonnes = []
for i in a :
    coordonnes.append(i)
v = 0
signe = 1
for i in coordonnes:
    if i in ["1","2","3","4","5","6","7","8","9","0"]:
        v = v*10 + int(i)
    elif i == "-":
        signe = -1
    elif i == ";" or i == ")":
        v = v * signe
        pts.append(v%mod)
        signe = 1
        v = 0
```
> mod is the modulo of the finite field

Then it created a ``Matrice_Gauss`` a list that is a matrix and add the coefficient 

```
Matrice_Gauss = []
for c in range(k):
    for l in range(k):
        Matrice_Gauss.append((pts[2*c]**l)%mod)
    Matrice_Gauss.append((pts[2*c+1])%mod)
```
The matrix is a way to simplify the equation system :
$$
\begin{cases} s+ax_1+bx_1²+cx_1³\dots= y_1\\ s+ax_2+bx_2²+cx_2³\dots = y_2\\ s+ax_3+bx_3²+cx_3³\dots = y_3\\s+ax_4+bx_4²+cx_4³\dots = y_4\\\dots \end{cases} (\bmod \ mod)
$$


The matrix now can be display with ``afficher(k,Matrice_Gauss)``
```
def afficher(k,Matrice_Gauss):
		for c in range(k):
			for l in range(k+1):
				print(Matrice_Gauss[c*(k+1)+l],end=" , ")
			print()
```

It now looks like : $[1,x_1,x_1²,x_1³,\dots y_1,1,x_2,x_2²,x_2³,\dots y_2,1,x_3,x_3²,x_3³,\dots y_3,\dots]$

And can be shown in a matrix like :
$$
\begin{pmatrix}
  1 & x_1 & x_1² & x_1³& \dots & y_1 \\
  1 & x_2 & x_2² & x_2³& \dots & y_2 \\
  1 & x_3 & x_3² & x_3³& \dots& y_3 \\
    \dots & \dots & \dots & \dots & \dots & \dots
\end{pmatrix} (\bmod \ mod)
$$
Which is a simplified way to write :

$$
\begin{pmatrix}
  1 & x_1 & x_1² & x_1³& \dots \\
  1 & x_2 & x_2² & x_2³ & \dots \\
  1 & x_3 & x_3² & x_3³ & \dots \\
  \dots& \dots& \dots& \dots&\dots
\end{pmatrix} \begin{pmatrix}
  s \\
  a \\
  b \\
\dots
\end{pmatrix} (\bmod \ mod)= \begin{pmatrix}
  y_1 \\
  y_2 \\
  y_3 \\
\dots
\end{pmatrix} (\bmod \ mod)
$$
>This Matrix is in reality a [Vandermonde Matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix)

In the matrix used by the script (the simplified one), there will always be $k$ columns and $k+1$ lines

>The variable ``c`` used in the script correspond to the columns number which start as 0 for the first
>
>The variable ``l`` used in the script correspond to the number which start as 0 for the first (in the explanation, ``l`` is replace by ``L`` so as not to confuse ``l`` with ``1``

Then the solve sequence start : it diagonalize the matrix in three phase

For each lines``L``it :

 1. Makes all the numbers in the line multiply by the modular inverse of``(L;L) ``
	 
	 To find the modular inverse of the number it solves :
	 $x * x^{-1} \bmod n = 1$
	 The algorithm solves it by testing every number but remember that this is not the best algorithm.
 
	 If something is divided by 0, the points are invalid and an error message is sent.
	As ``Matrix_Gauss`` is a one dimension list, the index correspond to ``c*(k+1)+L`` with ``c`` the column of the value, ``k``the number of lines and ``L`` the line of the value

	>Reminder ``c`` and ``L`` start at 0 not 1
	> If you look closer``k+1``corresponds to the number of value in a line

```
for colonneactive in range(k):         
	pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
	inverse_pivot = 0
	while (inverse_pivot * pivot)%mod != 1:
	    inverse_pivot += 1
	    if inverse_pivot == mod**2:
	        
	        return "Impossible"

	if pivot == 0:
	    impossibilite = 1
	    break
 			
```

So for the first line the matrix stay unchanged : this is because the value  ``(L;L)`` equals to $1$ so its divide all the numbers in this line by $1$ which doesn't change anything

$$
\begin{pmatrix}
  1 & x_1 & x_1² & x_1³& \dots & y_1 \\
  1 & x_2 & x_2² & x_2³& \dots & y_2 \\
  1 & x_3 & x_3² & x_3³& \dots & y_3 \\
    \dots & \dots & \dots& \dots & \dots & \dots
\end{pmatrix}
$$





 2. Find for each line ``line_to_change`` except the line ``L`` the coefficient ``coefcolumn`` so ``(L:L) * coefcolumn = ([line_to_change];L)`` which equals to``([line_to_change];L)`` because ``(L:L)`` now equals one and add for each column ``c`` 
  ``-(L;c)*coefcolumn``

```
for l in range(k+1):
                
    Matrice_Gauss[colonneactive*(k+1)+l] = (Matrice_Gauss[colonneactive*(k+1)+l]*inverse_pivot)%mod

for c in range(k):
    coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
    for l in range(k+1):
        if c != colonneactive:
            Matrice_Gauss[c*(k+1)+l]=(Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne)%mod
    		
```


After this step the matrix looks like :
$$
\begin{pmatrix}
  1 & x_1 & x_1² & x_1³&\dots& y_1 \\
  0 & x_2-x_1 & x_2²-x_1² & x_2³-x_1³&\dots & y_2-y_1 \\
  0 & x_3-x_1 & x_3²-x_1² & x_3³-x_1³& \dots & y_3-y_1 \\
    \dots & \dots & \dots &\dots & \dots & \dots
\end{pmatrix}
$$

Now steps 1 makes it look like :
$$
\begin{pmatrix}
  1 & x_1 & x_1² & x_1³& \dots& y_1 \\
  0 & 1 & \frac{x_2²-x_1²}{x_2-x_1} & \frac{x_2³-x_1³}{x_2-x_1} & \dots& \frac{y_2-y_1}{x_2-x_1}\\
  0 & x_3-x_1 & x_3²-x_1² & x_3³-x_1³ & \dots & y_3-y_1 \\
   \dots & \dots & \dots&\dots &\dots& \dots
\end{pmatrix}
$$

And step 2 makes it look like :


$\begin{pmatrix}
  1 & 0 & x_1²-\frac{x_2²-x_1²}{x_2-x_1}x_1 & x_1³-\frac{x_2³-x_1³}{x_2-x_1}x_1 &\dots& y_1-\frac{y_2-y_1}{x_2-x_1}x_1 \\
  0 & 1 & \frac{x_2²-x_1²}{x_2-x_1} & \frac{x_2³-x_1³}{x_2-x_1} & \dots & \frac{y_2-y_1}{x_2-x_1}\\
  0 & 0 & x_3²-x_1²-\frac{x_2²-x_1²}{x_2-x_1}(x_3-x_1) & x_3³-x_1³-\frac{x_2³-x_1³}{x_2-x_1}(x_3-x_1) &\dots& y_3-y_1-\frac{y_2-y_1}{x_2-x_1}(x_3-x_1) \\
    \dots & \dots & \dots& \dots & \dots & \dots
\end{pmatrix}$

At the end the matrix will look like :

$$
\begin{pmatrix}
  1 & 0 & 0 & 0 & \dots & v_1 \\
  0 & 1 & 0 & 0 & \dots & v_2 \\
  0 & 0 & 1 & 0 & \dots & v_3 \\
 0 & 0 & 0 & 1 & \dots & v_4 \\
    \dots & \dots & \dots& \dots & \dots & \dots
\end{pmatrix}
$$

> Note : $v$ refers to value

It has been diagonalized, so the matrix correspond to :

$$
\begin{pmatrix}
  1 & 0 & 0 & 0 & \dots \\
  0 & 1 & 0 & 0 & \dots  \\
  0 & 0 & 1 & 0 & \dots  \\
 0 & 0 & 0 & 1 & \dots  \\
    \dots & \dots & \dots& \dots & \dots 
\end{pmatrix}\begin{pmatrix}
  s \\
  a \\
  b \\
  c\\
\dots
\end{pmatrix} = \begin{pmatrix}
  v_1 \\
  v_2 \\
  v_3 \\
  v_4\\
\dots
\end{pmatrix}
$$



$$
\iff
\begin{pmatrix}
  1s & 0a & 0b& 0c & \dots \\
  0s & 1a & 0b & 0c & \dots  \\
  0s & 0a & 1b & 0c & \dots  \\
 0s & 0a & 0b & 1c & \dots  \\
    \dots & \dots & \dots& \dots & \dots 
\end{pmatrix} = \begin{pmatrix}
  v_1 \\
  v_2 \\
  v_3 \\
  v_4\\
\dots
\end{pmatrix} \qquad \qquad \qquad
$$
In a equation system this equals to :
$$
\begin{cases} s= v_1\\ a = v_2\\ b = v_3\\c = v_4\\\dots \end{cases}
$$

Now that we have all the coefficients we can reconstitute the initial $f$ function.

```
print("La fonction est : f(x) = ",end="")
for c in range(k):
	if c == 0:
		print(Matrice_Gauss[k],end="",sep="")
	elif c == 1:
		print(" + ",Matrice_Gauss[(c+1)*(k+1)-1],"x",end="",sep="")
	else:
		print(" + ",Matrice_Gauss[(c+1)*(k+1)-1],"x^",c,end="",sep="")

print()
```





In [ ]:
#Matrice de Gauss

def afficher(k,Matrice_Gauss):
		for c in range(k):
			for l in range(k+1):
				print(Matrice_Gauss[c*(k+1)+l],end=" , ")
			print()



def solve_finite_field(k,mod):
    fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
    a = fichier.read()
    fichier.close()
    pts = []
    coordonnes = []
    for i in a :
        coordonnes.append(i)
    v = 0
    signe = 1
    for i in coordonnes:
        if i in ["1","2","3","4","5","6","7","8","9","0"]:
            v = v*10 + int(i)
        elif i == "-":
            signe = -1
        elif i == ";" or i == ")":
            v = v * signe
            pts.append(v%mod)
            signe = 1
            v = 0
    


    if k <= 0:
        return "Erreur, une des valeurs n'est pas valable"
    else:
        if len(pts)-2*k < 0:
            return "Les points fournis ne sont pas suffisants"
        for i in range(len(pts)-2*k):
            del pts[0]
        Matrice_Gauss = []
        for c in range(k):
            for l in range(k):
                Matrice_Gauss.append((pts[2*c]**l)%mod)
            Matrice_Gauss.append((pts[2*c+1])%mod)


        impossibilite = 0
        for colonneactive in range(k):
            
            pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
            inverse_pivot = 0
            while (inverse_pivot * pivot)%mod != 1:
                inverse_pivot += 1
                if inverse_pivot == mod**2:
                    
                    return "Impossible"

            if pivot == 0:
                impossibilite = 1
                break
            for l in range(k+1):
                
                Matrice_Gauss[colonneactive*(k+1)+l] = (Matrice_Gauss[colonneactive*(k+1)+l]*inverse_pivot)%mod

            for c in range(k):
                coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
                for l in range(k+1):
                    if c != colonneactive:
                        Matrice_Gauss[c*(k+1)+l]=(Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne)%mod
        
        if impossibilite == 1:
            return "Les points fournis ne sont pas suffisants"
        else:

            print("Matrice diagonalisée :")
            
            print("La fonction est : f(x) = ",end="")
            for c in range(k):
                if c == 0:
                    print(Matrice_Gauss[k],end="",sep="")
                elif c == 1:
                    print(" + ",Matrice_Gauss[(c+1)*(k+1)-1],"x",end="",sep="")
                else:
                    print(" + ",Matrice_Gauss[(c+1)*(k+1)-1],"x^",c,end="",sep="")
                    print("( modulo",mod,")")

            
        



In [ ]:
solve_finite_field(3,5)